#Speech Articulatory Coding (SPARC)

This lab provides hands-on experience in using Speech Articulatory Coding (SPARC) model.

SPARC is a neural encoding-decoding framework that encodes speech into 14 articulatory features (50Hz) with a disentangled speaker embedding. The individual channel in articulatory features corresponds to a physical articulator on the vocal tract, making the features interpretable, controllable and grounded. The decoder synthesizes those features back to speech audio. Please check the [paper](https://arxiv.org/abs/2406.12998) for detail.

In [ ]:
# Installation

!git clone https://github.com/Berkeley-Speech-Group/Speech-Articulatory-Coding.git

In [ ]:
!pip install -e /content/Speech-Articulatory-Coding/.

In [ ]:
import sys
sys.path.append("/content/Speech-Articulatory-Coding/")

In [ ]:
# audio samples
!ls /content/Speech-Articulatory-Coding/sample_audio

In [ ]:
from sparc import load_model
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

color_code = {"UL":matplotlib.colors.to_rgb("#EE3A5B"),
              "LL":matplotlib.colors.to_rgb("#FFD155"),
              "LI":matplotlib.colors.to_rgb("#959595"),
              "TT":matplotlib.colors.to_rgb("#43B962"),
              "TB":matplotlib.colors.to_rgb("#C44B9F"),
              "TD":matplotlib.colors.to_rgb("#0093B7"),
              "Loudness":matplotlib.colors.to_rgb("#FB754D"),
              "Pitch":matplotlib.colors.to_rgb("#FB754D"),}

def plot_art(ax, art, gap=5, skip_y=False,color=None,alpha=1.0, lw=2):
    yticks=[]
    ytick_labels=[]
    labels = ["UL","LL", "LI", "TT", "TB", "TD"]
    chidxs = []
    EMA_channel_label = ['TDX','TDY','TBX','TBY','TTX','TTY','LIX','LIY','ULX','ULY','LLX','LLY']
    channel_label = EMA_channel_label
    for l in labels:
        chidxs.append(EMA_channel_label.index(l+"X"))
        chidxs.append(EMA_channel_label.index(l+"Y"))


    for i,ch_i in enumerate(chidxs):
        ch_label = channel_label[ch_i]
        ytick_labels.append(ch_label)
        if ch_i < 12:
            art_name= ch_label[:2]
            if color is None:
                color = color_code[art_name]
            ax.plot(art[:,ch_i]-gap*i,color=color,alpha=alpha,lw=lw)
        else:
            art_name= ch_label
            if color is None:
                color = color_code[art_name]
            ax.plot(art[:,ch_i]-gap*i,color=color,alpha=alpha,lw=lw)
        yticks.append(-gap*i)
    if skip_y:
        ytick_labels = [""]*len(ytick_labels)

    ax.set_yticks(yticks,ytick_labels,fontsize=15)

    xticks= np.arange(0,len(art),50)
    xtick_labels= [f"{int(x*20/1000)}" for x in xticks]
    ax.set_xticks(xticks, xtick_labels,fontsize=15)
    ax.set_xlabel("Time (s)", fontsize=15)
    ax.set_xlim(0,len(art))
    return yticks

In [ ]:
coder = load_model("en", device= "cuda", use_penn=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Resynthesis

In [ ]:
wav_file = '/content/Speech-Articulatory-Coding/sample_audio/diva_babbling-84.wav'

In [ ]:
# EMA channel_label = ['TDX','TDY','TBX','TBY','TTX','TTY','LIX','LIY','ULX','ULY','LLX','LLY']
code = coder.encode(wav_file)

In [ ]:
#shapes
for name, values in code.items():
  print(f"{name}: {values.shape}")

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(7,8))
plot_art(ax, code['ema'][:400], color='C0', gap=6, alpha=1, lw=2)
plt.show()

In [ ]:
wav = coder.decode(**code)

In [ ]:
# resynthesized
ipd.display(ipd.Audio(wav, rate=coder.sr))


In [ ]:
# ground truth reference
wavt,tsr = sf.read(wav_file)
ipd.display(ipd.Audio(wavt, rate=tsr))

## Zero-shot Voice Conversion

In [ ]:
src_wav_file = "/content/Speech-Articulatory-Coding/sample_audio/sample1.wav"
trg_wav_file = "/content/Speech-Articulatory-Coding/sample_audio/sample2.wav"

In [ ]:
# source audio
src_wav,sr = sf.read(src_wav_file)
ipd.display(ipd.Audio(src_wav, rate=sr))

# target audio
trg_wav,sr = sf.read(trg_wav_file)
ipd.display(ipd.Audio(trg_wav, rate=sr))

In [ ]:
converted_wav = coder.convert(src_wav_file, trg_wav_file)

In [ ]:
# converted
ipd.display(ipd.Audio(converted_wav, rate=coder.sr))

In [ ]:
src_code = coder.encode(src_wav_file)
converted_code = coder.encode(converted_wav)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(7,8))
plot_art(ax, src_code['ema'][:400], color='C0', gap=6, alpha=1, lw=2)
plot_art(ax, converted_code['ema'][:400], color='C3', gap= 6, alpha=0.7, lw=1.5)
ax.plot([],[],label="Original source", color='C0', lw=2)
ax.plot([],[],label="After voice conversion", color='C3', lw=2,)
ax.legend(fontsize=15, handlelength=1.3,loc='lower right',bbox_to_anchor=(1.0,0.98), ncol=2, frameon=False)
plt.show()

## Record Yourself!

In [ ]:
# Helper function for recording from Colab. Adapted from https://colab.research.google.com/drive/14v6co8Ec1JUAVdvuX4D08zxbRM1LmtxQ#scrollTo=9y4bzwMsh05H.
# You will need to grant mic permission.
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5, file_name='recorded_audio.wav'):
  try:
    from google.colab import output
  except ImportError:
    print('No possible to import output from google.colab')
    return ''
  else:
    print('Recording')
    display(ipd.Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    print('Saving to', file_name)
    b = b64decode(s.split(',')[1])
    with open(file_name, 'wb') as f:
      f.write(b)
    return file_name

In [ ]:
# Recording from your laptop for 5 seconds. The file will be saved as 'recorded_audio.wav'.
# The file can be found in the navigator at the left panel, under the "content" folder.
# Download the files if you need, otherwise they will be deleted when the session ends.

record(sec=10, file_name='recorded_audio.wav')
!ffmpeg -y -i ./recorded_audio.wav recorded_audio.wav

In [ ]:
code = coder.encode('recorded_audio.wav')

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(7,8))
plot_art(ax, code['ema'][:400], color='C0', gap=6, alpha=1, lw=2)
plt.show()

In [ ]:
wav = coder.decode(**code)

In [ ]:
# resynthesized
ipd.display(ipd.Audio(wav, rate=coder.sr))


In [ ]:
# ground truth reference
wavt,tsr = sf.read('recorded_audio.wav')
ipd.display(ipd.Audio(wavt, rate=tsr))